Demonstrations for the theory of <a class="ProveItLink" href="theory.ipynb">proveit.logic.equality</a>
========

In [1]:
import proveit
%begin demonstrations

## Substitution

Equality is a fundamental concept of logic.  When two mathematical operations are equal, then one may be substituted for the other.  This is rule is defined by the `substitution` axiom:

In [2]:
from proveit.logic.equality  import substitution
from proveit import a, b, c, d, x, y, z, fx # we'll use these later
substitution

|- forall_{f, x, y | x = y} (f(x) = f(y))

The English translation of this axiom is: for any function $f$ and any $x, y$ such that $x=y$, $f(x) = f(y)$.  In other words, we may substitute $y$ for $x$ in any function whenever $x=y$.  The equality of $x$ and $y$ transfers to an equality between $f(x)$ and $f(y)$.  This is fundamental to the meaning of equality regardless of what $f$ does (as long as it can act on a single argument).  We may instantiate this axiom using any operation for $f$.  For example,

In [3]:
from proveit.logic import Not, Equals
substitution.instantiate({fx:Not(x), x:a, y:b}, assumptions=[Equals(a, b)])

{a = b} |- [not](a) = [not](b)

There are more convenient ways to apply this substitution rule than manual instantiation that was demonstrated in the previous input.  The `Equals` class has the `substitution`, `sub_left_side_into` and `sub_right_side_into` methods for conveniently applying substitution and its variants, as we will demonstrate below.  Each of these methods takes a `lambda_map` argument to provide a theory for the substitution -- what is being substituted and where.  A `lambda_map` can be an actual **Lambda** expression, an **InnerExpr** object, or if it is neither of these, it can be any other **Expression** for which the default will be to performa a global replacement.

In [4]:
from proveit import Lambda
from proveit.numbers import Add, frac, Exp
expr = Equals(a, Add(b, frac(c, d), Exp(c, d)))          

expr: a = (b + (c / d) + c^{d})

The `global_repl` static method of `Lambda` is useful for creating a global replacement lambda map.  Below, we create a map for replacing every occurence of $d$ in `expr` with anything else:

In [5]:
g_repl = Lambda.global_repl(expr, d)

g_repl: _a -> (a = (b + (c / _a) + c^{_a}))

We now use this lambda map to replace occurences of $d$ in `expr` with $y$:

In [6]:
d_eq_y = Equals(d, y)

d_eq_y: d = y

In [7]:
d_eq_y.substitution(g_repl, assumptions=[d_eq_y])

{d = y} |- (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{y}))

Or we can take advantage of that a global replacement is performed by default when a non-**Lambda** expression is provided as the "`lambda_map`".

In [8]:
d_eq_y.substitution(expr, assumptions=[d_eq_y])

{d = y} |- (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{y}))

Either way, the generated proof is the same:

In [9]:
d_eq_y.substitution(expr, assumptions=[d_eq_y]).proof()

step type	requirements	statement
0	instantiation	1, 2	{d = y} |- (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{y}))
	f(_a) : a = (b + (c / _a) + c^{_a}), x : d, y : y
1	axiom		|- forall_{f, x, y | x = y} (f(x) = f(y))
	proveit.logic.equality.substitution
2	assumption		{d = y} |- d = y

If we want to perform substitution for a specific inner expression, and not necessarily a global replacement, the `proveit._core_.expression.InnerExpr` class (aliased as `proveit.InnerExpr`) is extremely convenient.  It uses some Python tricks via implementing the `__getitem__` and `__getattr__` methods.  First, you create the `InnerExpr` object by calling the `inner_expr()` method on the top-level expression:

In [10]:
inner_expr = expr.inner_expr()

inner_expr: {lambda:[_a -> _a], ${_{-}a}$:(a = (b + (c / d) + c^{d}))}

The `InnerExpr` object displays itself with two important pieces of information: the lambda map that it represents and the inner expression of the top-level expression that would be replaced by this lambda map.  The point is that we will be able to "dig" in to inner expressions of the top-level expression via accessing sub-expression attributes.  For example, we next will "dig" into the "right hand side" (`rhs`) of the master expression: 

In [11]:
inner_expr = inner_expr.rhs

inner_expr: {lambda:[_a -> (a = _a)], ${_{-}a}$:(b + (c / d) + c^{d})}

By accessing the `rhs` attribute, we created a new `InnerExpr` object that has the same top-level expression as the original but has a new current inner expression.  Note that the `InnerExpr` class does not know anything about the `rhs` attribute itself; it is relying on the fact that the previous sub-expression has this attribute.  The `InnerExpr` class also has tricks for getting an sub-expression with an index (or key):

In [12]:
inner_expr = inner_expr.operands[1]

inner_expr: {lambda:[_a -> (a = (b + _a + c^{d}))], ${_{-}a}$:(c / d)}

Now will "dig" down to the denominator of $\frac{c}{d}$ and show how we use the `InnerExpr` class to replace a particular occurence of $d$ rather than a global replacement:

In [13]:
inner_expr = inner_expr.denominator

inner_expr: {lambda:[_a -> (a = (b + (c / _a) + c^{d}))], ${_{-}a}$:d}

In [14]:
d_eq_y.substitution(inner_expr, assumptions=[d_eq_y])

{d = y} |- (a = (b + (c / d) + c^{d})) = (a = (b + (c / y) + c^{d}))

Let us demonstrate this technique again, replacing the other occurrence of $d$.  This time we do this more succinctly, without the extra pedogogial steps:

In [15]:
d_eq_y.substitution(expr.inner_expr().rhs.operands[2].exponent, assumptions=[d_eq_y])

{d = y} |- (a = (b + (c / d) + c^{d})) = (a = (b + (c / d) + c^{y}))

The `substition` method, that we demonstrated above, is a direct application of the `substitution` axiom.  It proves the equality between some $f(x)$ and some $f(y)$.  We often will want to take a shortcut to perform a statement substitution in which we prove some $P(y)$ is true assuming that $P(x)$ is true and $P(x) = P(y)$.  For this, we have the `sub_right_side_into` and `sub_left_side_into` methods.

If the expression that we want to substitute in is on the right hand side of the `Equals` object playing the role of $x=y$, then we use `sub_right_side_into`: 

In [16]:
d_eq_y.sub_right_side_into(g_repl, assumptions=[d_eq_y,expr])

{a = (b + (c / d) + c^{d}), d = y} |- a = (b + (c / y) + c^{y})

We can also take advantage of the global replacement default and provide a non-**Lambda** **Expression**.

In [17]:
d_eq_y.sub_right_side_into(expr, assumptions=[d_eq_y,expr])

{a = (b + (c / d) + c^{d}), d = y} |- a = (b + (c / y) + c^{y})

If the expression that we want to substitute in is on the left hand side of the `Equals` object playing the role of $x=y$, then we use `sub_left_side_into`: 

In [18]:
y_eq_d = Equals(y, d)

y_eq_d: y = d

In [19]:
y_eq_d.sub_left_side_into(g_repl, assumptions=[y_eq_d,expr])

{a = (b + (c / d) + c^{d}), y = d} |- a = (b + (c / y) + c^{y})

Again, we can provide a non-**Lambda** **Expression** to do a simple global replacement.

In [20]:
y_eq_d.sub_left_side_into(expr, assumptions=[y_eq_d,expr])

{a = (b + (c / d) + c^{d}), y = d} |- a = (b + (c / y) + c^{y})

The proof uses a theorem the relies upon the `substitution` axiom, rather than using the `substition` axiom directly:

In [21]:
y_eq_d.sub_left_side_into(expr, assumptions=[y_eq_d,expr]).proof()

step type	requirements	statement
0	instantiation	1, 2, 3	{a = (b + (c / d) + c^{d}), y = d} |- a = (b + (c / y) + c^{y})
	P(_a) : a = (b + (c / _a) + c^{_a}), x : y, y : d
1	theorem		|- forall_{P, x, y | P(y), x = y} P(x)
	proveit.logic.equality.sub_left_side_into
2	assumption		{a = (b + (c / d) + c^{d})} |- a = (b + (c / d) + c^{d})
3	assumption		{y = d} |- y = d

## Reflexivity, symmetry, and transitivity

Reflexivity, symmetry, and transitivity are also fundamental properties of equality, in addition to the ability to perform substitution.  Reflexivity is the fact that any mathematical object is equal to itself.  Symmetry is the fact that $x = y$ and $y = x$ are equivalent (either both of these are true or both of these false).  Transitivity is the ability to derive $x=z$ from $x=y$ and $y=z$.  These are all axioms.

In [22]:
from proveit.logic.equality  import equals_reflexivity, equals_symmetry, equals_transitivity

In [23]:
equals_reflexivity

|- forall_{x} (x = x)

In [24]:
equals_symmetry

|- forall_{x, y} ((y = x) = (x = y))

In [25]:
equals_transitivity

|- forall_{x, y, z | x = y, y = z} (x = z)

`equals_reversal` is a useful theorem for applying the symmetry property of equality:

In [26]:
from proveit.logic.equality import equals_reversal
equals_reversal # y=x derives from x=y

|- forall_{x, y | x = y} (y = x)

These three properties are applied automatically for `Equals` objects

Reflexivity is concluded automatically:

In [27]:
Equals(a, a).prove()

|- a = a

In [28]:
Equals(a, a).prove().proof()

step type	requirements	statement
0	instantiation	1	|- a = a
	x : a
1	axiom		|- forall_{x} (x = x)
	proveit.logic.equality.equals_reflexivity

Symmetric statements are derived as side-effects.  Note that the `Judgment.derive_side_effects` method employs a mechanism to prevent infinite recursion or this would not be possible (it would continually go back and forth, proving $y=x$ from $x=y$ then $x=y$ from $y=x$, ad infinitum)

In [29]:
a_eq_b = Equals(a, b)

a_eq_b: a = b

In [30]:
Equals(b, a).prove([a_eq_b])

{a = b} |- b = a

The reversed form may also be derived explicitly via `derive_reversed`.  The proof is the same.

In [31]:
a_eq_b.prove([a_eq_b]).derive_reversed()

{a = b} |- b = a

Transitivity derivations are attempted with automation via the `conclude_via_transitivity` method from the `TransitiveRelation` superclass of `Equals`.  This performs a breadth-first, bidirectional search (meeting in the middle from both ends) over the space of **Judgment** objects representing equality and using appropriate assumptions.  This is therefore reasonably efficient.  Efficiency should not really be an issue, anyways, as long as proofs for each theorem are relatively small.  A long proof should be broken up into several smaller proofs for lemma-like theorems.  In that case (in the setting of small proofs), the space of **Judgment**s will be small and this search algorithm will have ample efficiency. 

In [32]:
# We'll make this interesting by reversing some of the equations in the chain.
c_eq_b = Equals(c, b) 

c_eq_b: c = b

In [33]:
d_eq_c = Equals(d, c)

d_eq_c: d = c

In [34]:
d_eq_z = Equals(d, z)

d_eq_z: d = z

In [35]:
a_eq_z = Equals(a, z).conclude_via_transitivity(assumptions=[a_eq_b, c_eq_b, d_eq_c, d_eq_z])

a_eq_z: {a = b, c = b, d = c, d = z} |- a = z

The `apply_transivity` method applies the transitivity relation explicity. 

In [36]:
a_eq_b.prove([a_eq_b]).apply_transitivity(c_eq_b, assumptions=[c_eq_b])

{a = b, c = b} |- a = c

## Evaluation

An evaluation is an equality in which the right hand side is an irreducible value (specifically, an instance of `proveit.logic.irreducible_value.IrreducibleValue`, aliased as `proveit.logic.IrreducibleValue`).  An irreducible value represents a mathematical object in its simplest form.  It cannot be reduced.  The evaluation of an `IrreducibleValue` is itself.  When an evaluation is proven, the associated **Judgment** is stored for future reference in `Equals.evaluations` for making other evaluations.

`TRUE` ($\top$) and `FALSE` ($\bot$) are both `IrreducibleValue`s:

In [37]:
from proveit.logic import TRUE, FALSE, IrreducibleValue

In [38]:
isinstance(TRUE, IrreducibleValue)

True

In [39]:
isinstance(FALSE, IrreducibleValue)

True

In [40]:
TRUE.evaluation()

|- TRUE = TRUE

An `IrreducibleValue` should implement the `eval_equality` and `not_equal` methods to prove equality relations with other `IrreducibleValue`s as appropriate.

In [41]:
Equals(Equals(FALSE, TRUE), FALSE).derive_contradiction([Equals(FALSE, TRUE)])

{FALSE = TRUE} |- FALSE

In [42]:
TRUE.eval_equality(TRUE)

|- (TRUE = TRUE) = TRUE

In [43]:
TRUE.eval_equality(FALSE)

|- (TRUE = FALSE) = FALSE

In [44]:
TRUE.not_equal(FALSE)

|- TRUE != FALSE

A proven expression will evaluate to `TRUE`.

In [45]:
a_eq_T = a.evaluation([a])

a_eq_T: {a} |- a = TRUE

A disproven expression will evaluate to `FALSE`.

In [46]:
a_eq_F = a.evaluation([Not(a)])

a_eq_F: {[not](a)} |- a = FALSE

If the expression to be evaluated is known to be equal to an expression that has already been evaluated, transitivity will automatically be applied.

In [47]:
b_eq_F = b.evaluation([Equals(b, a), Not(a)])

b_eq_F: {b = a, [not](a)} |- b = FALSE

In [48]:
b_eq_F.proof()

step type	requirements	statement
0	instantiation	1, 2, 3	{b = a, [not](a)} |- b = FALSE
	x : b, y : a, z : FALSE
1	axiom		|- forall_{x, y, z | x = y, y = z} (x = z)
	proveit.logic.equality.equals_transitivity
2	assumption		{b = a} |- b = a
3	instantiation	4, 5	{[not](a)} |- a = FALSE
	A : a
4	axiom		|- forall_{A | [not](A)} (A = FALSE)
	proveit.logic.booleans.negation.negation_elim
5	assumption		{[not](a)} |- [not](a)

When evaluating an expression, an evaluation of the operands will be attempted.  The operation class is responsible for overridding the `evaluation` method, or `do_reduced_evaluation` (called by the default `Expression.evaluation` method that first reduces the operation by evaluating its operands) in order to properly treat the operation applied to irreducible values or generate a more efficient proof as appropriate.

In [49]:
from proveit.logic import Or, Not, in_bool

In [50]:
nested_eval = Or(a, Not(a)).evaluation([a])

nested_eval: {a} |- (a or [not](a)) = TRUE

## Boolean equality

Equality with `TRUE` ($\top$) or `FALSE` ($\bot$) has special logical consequences.  The `Equals` object has automation capabilities to treat these special kinds of equations.

In [51]:
TRUE

TRUE

In [52]:
FALSE

FALSE

Proofs via boolean equality are automatic via `Equals.deduce_side_effects`:

In [53]:
a.prove([Equals(a, TRUE)])

{a = TRUE} |- a

In [54]:
Not(b).prove([Equals(b, FALSE)])

{b = FALSE} |- [not](b)

Going the other direction, boolean equalities are proven automatically via `Equals.conclude`:

In [55]:
Equals(x, TRUE).prove([x])

{x} |- x = TRUE

In [56]:
Equals(TRUE, y).prove([y])

{y} |- TRUE = y

In [57]:
Equals(c, FALSE).prove([Not(c)])

{[not](c)} |- c = FALSE

In [58]:
Equals(FALSE, c).prove([Not(c)])

{[not](c)} |- FALSE = c

When something is equal to `FALSE` and known to be true, there is a contradiction.  That is, `FALSE` is a consequence.  The `derive_contradiction` method of `Equals` can be used to prove such a contradiction:

In [59]:
contradiction = Equals(a, FALSE).derive_contradiction([a, Equals(a, FALSE)])

contradiction: {a, a = FALSE} |- FALSE

In [60]:
contradiction.proof()

step type	requirements	statement
0	instantiation	1, 2, 3	{a, a = FALSE} |- FALSE
	A : a
1	theorem		|- forall_{A | A, A = FALSE} FALSE
	proveit.logic.equality.contradiction_via_falsification
2	assumption		{a} |- a
3	assumption		{a = FALSE} |- a = FALSE

Contradictions are useful in making contradiction proofs (reductio ad absurdum).  The `affirm_via_contradiction` and `deny_via_contradiction` methods are useful in making such a proof.  They both use `derive_contradiction`.  For example:

In [61]:
from proveit.logic import Implies, in_bool
not_a__truth = Equals(a, FALSE).affirm_via_contradiction(Not(a), [Implies(a, Equals(a, FALSE)), in_bool(a)])

not_a__truth: {a in BOOLEAN, a => (a = FALSE)} |- [not](a)

In [62]:
not_a__truth.proof().num_steps()

13

It is more efficient to use `deny_via_contradiction` when proving the negation of something.  Here we prove the same as above but in fewer steps.

In [63]:
not_b__truth = Equals(b, FALSE).deny_via_contradiction(b, [Implies(b, Equals(b, FALSE)), in_bool(b)])

not_b__truth: {b in BOOLEAN, b => (b = FALSE)} |- [not](b)

In [64]:
not_b__truth.proof().num_steps()

9

In [65]:
not_b__truth.proof()

step type	requirements	statement
0	instantiation	1, 2, 3	{b in BOOLEAN, b => (b = FALSE)} |- [not](b)
	A : b
1	axiom		|- forall_{A in BOOLEAN | A => FALSE} [not](A)
	proveit.logic.booleans.implication.denial_via_contradiction
2	assumption		{b in BOOLEAN} |- b in BOOLEAN
3	deduction	4	{b => (b = FALSE)} |- b => FALSE
4	instantiation	5, 8, 6	{b, b => (b = FALSE)} |- FALSE
	A : b
5	theorem		|- forall_{A | A, A = FALSE} FALSE
	proveit.logic.equality.contradiction_via_falsification
6	modus ponens	7, 8	{b => (b = FALSE), b} |- b = FALSE
7	assumption		{b => (b = FALSE)} |- b => (b = FALSE)
8	assumption		{b} |- b

## Equality and sets

All equality expressions are in the boolean set by the `equality_in_bool` axiom.  That is, given any two mathematical objects, they are either equal or not (even nonsense is either the same nonsense or different nonsense).

In [66]:
eq_in_bool = Equals(a, b).deduce_in_bool()

eq_in_bool: |- (a = b) in BOOLEAN

A singleton is a set with one element.  If $x=c$, then $x$ is in the singleton set of $\{c\}$:

In [67]:
in_singleton_truth = Equals(x, c).derive_is_in_singleton([Equals(x, c)])

in_singleton_truth: {x = c} |- x in {c}

## Not equals

The `NotEquals` operation is a more way of expressing that two mathematical objects are not equal to each other.

In [68]:
from proveit.logic import NotEquals

In [69]:
a_neq_b = NotEquals(a, b)

a_neq_b: a != b

In [70]:
a_neq_b.definition()

|- (a != b) = [not](a = b)

From $\lnot (a = b)$ one can derive $a \neq b$ and vice-versa, folding and unfolding the `NotEquals`.

In [71]:
a_neq_b.prove([Not(a_eq_b)])

{[not](a = b)} |- a != b

In [72]:
a_neq_b.prove([Not(a_eq_b)]).proof()

step type	requirements	statement
0	instantiation	1, 2	{[not](a = b)} |- a != b
	x : a, y : b
1	theorem		|- forall_{x, y | [not](x = y)} (x != y)
	proveit.logic.equality.fold_not_equals
2	assumption		{[not](a = b)} |- [not](a = b)

In [73]:
Not(a_eq_b).prove([a_neq_b])

{a != b} |- [not](a = b)

In [74]:
Not(a_eq_b).prove([a_neq_b]).proof()

step type	requirements	statement
0	instantiation	1, 2	{a != b} |- [not](a = b)
	x : a, y : b
1	theorem		|- forall_{x, y | x != y} [not](x = y)
	proveit.logic.equality.unfold_not_equals
2	assumption		{a != b} |- a != b

`NotEquals` also has a symmetry property which can be applied directly.  It can be proven  through automation.

In [75]:
NotEquals(b, a).prove([a_neq_b])

{a != b} |- b != a

Or explicitly via `derive_reversed`

In [76]:
a_neq_b.prove([a_neq_b]).derive_reversed()

{a != b} |- b != a

If two objects are both equal and not equal, there is a contradiction.

In [77]:
neq_contradiction = a_neq_b.derive_contradiction([a_neq_b, a_eq_b])

neq_contradiction: {a = b, a != b} |- FALSE

In [78]:
a_neq_a = NotEquals(a, a)

a_neq_a: a != a

In [79]:
b_from_contradiction = a_neq_a.affirm_via_contradiction(b, [Implies(Not(b), a_neq_a), in_bool(b)])

b_from_contradiction: {b in BOOLEAN, [not](b) => (a != a)} |- b

In [80]:
%end demonstrations